In [ ]:
import dask.dataframe as dd
from dask.distributed import Client
import numpy as np
import pandas as pd
import sys, os

# Khởi tạo Dask client để tối ưu hiệu suất
client = Client()

file_path = "remapping_dataset_2type.csv"
output_files = ["test.csv", "file1.csv", "file2.csv", "file3.csv"]
# Tỉ lệ chia (3:2:1)

# Cấu hình
chunksize = 100000  # Đọc từng phần (tuỳ vào RAM)
temp_dir = "FL_Dataset/"  # Thư mục lưu file tạm

output_files = [temp_dir + output_file for output_file in output_files]
print(output_files)

header_saved=False
for chunk in pd.read_csv(file_path, chunksize=chunksize):
    df_1 = chunk.sample(frac=0.2, random_state=42)
    df_rest = chunk.drop(df_1.index)
    df_2 = df_rest.sample(frac=1/4, random_state=42)
    df_rest1 = df_rest.drop(df_2.index)
    df_3 = df_rest1.sample(frac=0.5, random_state=42)
    df_4 = df_rest1.drop(df_3.index)

    df_1.to_csv(output_files[0], mode="a", header=not header_saved, index=False)
    df_2.to_csv(output_files[1], mode="a", header=not header_saved, index=False)
    df_3.to_csv(output_files[2], mode="a", header=not header_saved, index=False)
    df_4.to_csv(output_files[3], mode="a", header=not header_saved, index=False)
    header_saved = True

/home/th/miniconda3/envs/tf/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46485 instead
  warnings.warn(


['FL_Dataset/test.csv', 'FL_Dataset/file1.csv', 'FL_Dataset/file2.csv', 'FL_Dataset/file3.csv']


## Split parquet

In [7]:
import os
import dask.dataframe as dd
p_dir = [ "C:/Users/hoang/FileCSV_DACN_2025/parquet_shuffled_IoT23", "/mnt/c/Users/hoang/FileCSV_DACN_2025/parquet_shuffled_IoT23"]
s_dir = [ "C:/Users/hoang/FileCSV_DACN_2025/fl_IoT23/", "/mnt/c/Users/hoang/FileCSV_DACN_2025/fl_IoT23/"]
s_f = [f"file{index}.csv" for index in range(3)]

p_dir = dir[0] if os.name == 'nt' else p_dir[1]
s_dir = dir[0] if os.name == 'nt' else s_dir[1]

s_f = [s_dir+f for f in s_f]
print("FL Files: ", s_f)

if os.path.exists(s_dir) ==False:
    os.makedirs(s_dir)
    
df = dd.read_parquet(p_dir)
print("Partitions Number: ", df.npartitions)

header = True
for partition in df.partitions:
    partition = partition.compute()
    df1 = partition.sample(frac = 0.2, random_state=42)
    tempdf = partition.drop(df1.index)
    df2 = tempdf.sample(frac=0.375, random_state=42)
    df3 = tempdf.drop(df2.index)
    
    df1.to_csv(s_f[0], mode='a', index=False, header=header)
    df2.to_csv(s_f[1], mode='a', index=False, header=header)
    df3.to_csv(s_f[2], mode='a', index=False, header=header)
    header = False    

FL Files:  ['/mnt/c/Users/hoang/FileCSV_DACN_2025/fl_IoT23/file0.csv', '/mnt/c/Users/hoang/FileCSV_DACN_2025/fl_IoT23/file1.csv', '/mnt/c/Users/hoang/FileCSV_DACN_2025/fl_IoT23/file2.csv']
Partitions Number:  862


In [3]:
import os
import dask.dataframe as dd 
p_dir = [ "C:/Users/hoang/FileCSV_DACN_2025/parquet_shuffled_IoT23", "/mnt/c/Users/hoang/FileCSV_DACN_2025/parquet_shuffled_IoT23"]
s_dir = [ "C:/Users/hoang/FileCSV_DACN_2025/fl_IoT23/", "/mnt/c/Users/hoang/FileCSV_DACN_2025/fl_IoT23/"]

s_dir = s_dir[0] if os.name == 'nt' else s_dir[1]
s_f = [s_dir+f"file{index}.csv" for index in range(3)]
p_dir = [ s_dir+f"parquet_f{index}" for index in range(3)]
print(p_dir)

for index, f in enumerate(s_f):
    print("Processing file ", f)
    df = dd.read_csv(f, blocksize= "256MB")
    df.to_parquet(p_dir[index])

['C:/Users/hoang/FileCSV_DACN_2025/fl_IoT23/parquet_f0', 'C:/Users/hoang/FileCSV_DACN_2025/fl_IoT23/parquet_f1', 'C:/Users/hoang/FileCSV_DACN_2025/fl_IoT23/parquet_f2']
Processing file  C:/Users/hoang/FileCSV_DACN_2025/fl_IoT23/file0.csv
Processing file  C:/Users/hoang/FileCSV_DACN_2025/fl_IoT23/file1.csv
Processing file  C:/Users/hoang/FileCSV_DACN_2025/fl_IoT23/file2.csv
